<a href="https://colab.research.google.com/github/KevinTheRainmaker/AI_Tutorials/blob/main/Transfer_Learning_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classify IMDB Reviews

## 0. Intro

In [2]:
!pip install -q tensorflow-hub
!pip install -q tfds-nightly
# pip install tensorflow-datasets : 몇 개월마다 릴리스되는 안정적인 버전
# pip install tfds-nightly : 매일 릴리스되는 최신 데이터 세트

In [3]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np

print(tf.__version__)
print(hub.__version__)

2.8.2
0.12.0


In [4]:
# check GPU
print('GPU', "On" if tf.config.experimental.list_physical_devices("GPU") else "Off")

GPU On


## 1. Data

In [28]:
train_data, val_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
) 

In [29]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

15000
10000
25000


In [30]:
train_example_batch, train_label_batch = next(iter(train_data.batch(10)))
print(train_example_batch)
print(train_label_batch )

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

## 2. Model

In [17]:
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.7657859 , -3.882232  ,  3.913424  , -1.5557289 , -3.3362343 ,
        -1.7357956 , -1.9954445 ,  1.298955  ,  5.081597  , -1.1041285 ,
        -2.0503852 , -0.7267516 , -0.6567596 ,  0.24436145, -3.7208388 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489715 , -1.1315986 ],
       [ 1.8804485 , -2.5852385 ,  3.4066994 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.7855542 ,  1.3874227 ,  3.8476458 , -0.9256539 ,
        -1.896706  ,  1.2113281 ,  0.11474716,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015903 , -1.6390051 ],
       [ 0.71152216, -0.63532174,  1.7385626 , -1.1168287 , -0.54515934,
        -1.1808155 ,  0.09504453,  1.4653089 ,  0.66059506,  0.79308075,
        -2.2268343 ,  0.07446616, -1.4075902 , -0.706454  , -1.907037  ,
         1.4419788 ,  1.9551864 , -0.42660046, -2.8022065 ,  0.43727067]],
      dtype=float32)>

In [18]:
model = keras.Sequential()

model.add(hub_layer)
model.add(keras.layers.Dense(16, activation = 'relu'))
model.add(keras.layers.Dense(1, activation = 'sigmoid'))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [21]:
history = model.fit(train_data.shuffle(10_000).batch(512),
                    epochs=20,
                    validation_data=val_data.batch(512))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


30/30 [==============================] - 5s 51ms/step - loss: 0.9450 - accuracy: 0.4763 - val_loss: 0.7530 - val_accuracy: 0.5237
Epoch 2/20
30/30 [==============================] - 2s 48ms/step - loss: 0.6925 - accuracy: 0.5765 - val_loss: 0.6317 - val_accuracy: 0.6473
Epoch 3/20
30/30 [==============================] - 2s 46ms/step - loss: 0.6009 - accuracy: 0.6796 - val_loss: 0.5778 - val_accuracy: 0.7062
Epoch 4/20
30/30 [==============================] - 2s 47ms/step - loss: 0.5508 - accuracy: 0.7315 - val_loss: 0.5401 - val_accuracy: 0.7396
Epoch 5/20
30/30 [==============================] - 2s 48ms/step - loss: 0.5135 - accuracy: 0.7610 - val_loss: 0.5094 - val_accuracy: 0.7605
Epoch 6/20
30/30 [==============================] - 2s 48ms/step - loss: 0.4796 - accuracy: 0.7869 - val_loss: 0.4818 - val_accuracy: 0.7814
Epoch 7/20
30/30 [==============================] - 2s 46ms/step - loss: 0.4478 - accuracy: 0.8072 - val_loss: 0.4548 - val_accuracy: 0.7981
Epoch 8/20
30/30 [======

In [35]:
results = model.evaluate(test_data.batch(512))

49/49 [==============================] - 2s 39ms/step - loss: 0.3166 - accuracy: 0.8658


In [39]:
for metrics, value in zip(model.metrics_names, results):
  print('Metrics: %s / Value: %.3f' % (metrics, value))

Metrics: loss / Value: 0.317
Metrics: accuracy / Value: 0.866
